In [14]:
import pandas as pd
from bokeh.charts import output_notebook, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.layouts import row
from bokeh.plotting import figure

output_notebook() 


Loading BokehJS ...

In [15]:
dataset = pd.read_csv("data.csv",index_col=0,parse_dates=True)


rich_weekday = lambda c: -1 if c["holiday"] else c["weekday"]

dataset.workingday = dataset.workingday==1
dataset.holiday = dataset.holiday==1
dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["season_s"] = dataset.season.map({1:"Printemps",2:"Ete",3:"Automne",4:"Hiver"})
dataset["weather_s"] = dataset.weather.map({1:"clear",2:"cloudy",3:"light rain",4:"crappy"})
dataset["rich_weekday"] = dataset.apply( rich_weekday, axis=1)

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
label = dataset["registered"] 
del dset["count"], dset["registered"], dset["casual"]
dset["hour"] =  dset.index.hour
dset["weekday"] = dset.index.weekday
#dset["hour_weekend"] = dset["hour"] * dataset.weekday.map(lambda i: 0 if (i in [6,7]) else 1)
#dset["hour_workweek"] = dset["hour"] * dataset.weekday.map(lambda i: 1 if (i in [6,7]) else 0)   
dset["h_wd"] = dset.weekday*100+dataset.hour

dset_scaled = preprocessing.scale(dset)
pca = PCA(n_components="mle")

dt = DecisionTreeRegressor(max_depth=25)
clf = AdaBoostRegressor(n_estimators=100,base_estimator=dt)
#clf = GradientBoostingRegressor(n_estimators=300,max_depth=4) 
#clf = Ridge(normalize=True, alpha=.1, tol=1e-15)
estimators = [('reduce_dim', pca), ('clf',clf)]
pipe = Pipeline(estimators)
#scores = cross_val_score(clf, dset, label,cv=20,verbose=1,scoring="neg_mean_absolute_error",n_jobs=6)
scores = cross_val_score(clf, dset, label,cv=5,verbose=1,scoring="neg_mean_squared_error",n_jobs=6)                          
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print scores

Avg dist to estimate: -9188.89 (+/- 3005.81)
[-10447.38953034  -9465.7956838   -6828.79244279  -8236.88622945
 -10965.60867903]


[Parallel(n_jobs=6)]: Done   5 out of   5 | elapsed:    1.9s finished


In [19]:
estimator = pipe.fit(dset,label)
full_dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
full_dset["hour"] =  dset.index.hour
full_dset["weekday"] = dset.index.weekday
full_dset["predicted"] = estimator.predict(dset)

In [20]:
col = "predicted"
gg =  full_dset.groupby(["weekday","hour"])[col].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y=col,color="weekday")

show(b)

In [21]:
from sklearn.model_selection import GridSearchCV
print clf.get_params()
#param_grid = [{'learning_rate':[1],'base_estimator__max_depth':[2,5,10],'n_estimators':[10,100,500,1000,5000]}]
param_grid = [{'learning_rate':[1],'max_depth':[2,5,10,20,30],'n_estimators':[10,100,500,1000,5000]}]
gscv = GridSearchCV(
    estimator = clf,
    param_grid=param_grid,
    scoring="r2",
    cv=5,
    verbose=2,
    n_jobs=6
)

f = gscv.fit(dset, label)

print f.best_estimator_
print f.best_score_

{'loss': 'linear', 'base_estimator__min_samples_split': 2, 'base_estimator__max_depth': 25, 'learning_rate': 1.0, 'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'), 'base_estimator__criterion': 'mse', 'base_estimator__max_features': None, 'base_estimator__random_state': None, 'n_estimators': 100, 'base_estimator__min_weight_fraction_leaf': 0.0, 'random_state': None, 'base_estimator__splitter': 'best', 'base_estimator__max_leaf_nodes': None, 'base_estimator__min_samples_leaf': 1, 'base_estimator__presort': False, 'base_estimator__min_impurity_split': 1e-07}
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] n_estimators=10, learning_rate=1, max_depth=2 ...................
[CV] n_estimators=10, learning_rate=1, max_depth=2 

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/home/avignon/.local/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

...................................................

In [22]:
from sklearn import svm
from sklearn import preprocessing
dset_scaled = preprocessing.scale(dset)
clf = svm.SVR(kernel='rbf',C=1,verbose=True, cache_size=1000)
#clf = svm.SVR(C=1e3,verbose=True)
scores = cross_val_score(clf, dset, label,cv=10,verbose=3,scoring="neg_median_absolute_error",n_jobs=6)
scores.mean()  
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[LibSVM][CV] ............................... , score=-85.911519, total=  10.5s
[CV]  ................................................................
[LibSVM][CV] ............................... , score=-85.350186, total=  10.6s
[CV]  ................................................................
[LibSVM][CV] ............................... , score=-79.613015, total=  11.0s
[CV]  ................................................................
[LibSVM][CV] ............................... , score=-74.731232, total=  11.1s
[CV]  .......................................

[Parallel(n_jobs=6)]: Done   3 out of  10 | elapsed:   11.1s remaining:   25.9s


[LibSVM][CV] ............................... , score=-78.240129, total=  11.7s
[LibSVM][CV] .............................. , score=-101.579633, total=   8.3s
[LibSVM][CV] .............................. , score=-102.405174, total=   8.3s


[Parallel(n_jobs=6)]: Done   7 out of  10 | elapsed:   18.9s remaining:    8.1s


[LibSVM][CV] .............................. , score=-103.467351, total=   8.3s
[LibSVM][CV] .............................. , score=-101.243150, total=   9.0s
Avg dist to estimate: -88.96 (+/- 22.53)


[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:   20.2s finished


In [ ]:
cross_val_score?

In [ ]:
dataset["count"].describe()

In [12]:
from tpot import TPOTRegressor
pipeline_optimizer = TPOTRegressor(generations=5, population_size=20, num_cv_folds=5, random_state=42, verbosity=2)
#(fits by default on lean squared errror)
label = dataset["registered"] 
dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
del dset["count"], dset["registered"], dset["casual"]
dset["hour"] =  dset.index.hour
dset["weekday"] = dset.index.weekday
#dset["hour_weekend"] = dset["hour"] * dataset.weekday.map(lambda i: 0 if (i in [6,7]) else 1)
#dset["hour_workweek"] = dset["hour"] * dataset.weekday.map(lambda i: 1 if (i in [6,7]) else 0)   
dset["h_wd"] = dset.weekday*100+dataset.hour

pipeline_optimizer.fit(dset, label)
print(pipeline_optimizer.score(dset, label))
pipeline_optimizer.export('tpot_exported_pipeline.py')

Optimization Progress:  17%|█▋        | 20/120 [10:20<53:12, 31.92s/pipeline]  n Progress:   1%|          | 1/120 [01:00<1:59:31, 60.27s/pipeline]          

Generation 1 - Current best internal CV score: 9186.9518442


Optimization Progress:  24%|██▍       | 29/120 [17:10<1:19:46, 52.59s/pipeline]timization Progress:  18%|█▊        | 22/120 [11:42<1:00:31, 37.06s/pipeline]          Optimization Progress:  25%|██▌       | 30/120 [17:10<1:13:32, 49.03s/pipeline]

Timeout during evaluation of pipeline #30. Skipping to the next pipeline.


Optimization Progress:  29%|██▉       | 35/120 [27:12<1:15:15, 53.13s/pipeline]Optimization Progress:  27%|██▋       | 32/120 [19:32<1:19:11, 54.00s/pipeline]          Optimization Progress:  30%|███       | 36/120 [27:12<2:59:17, 128.07s/pipeline]

...

Generation 4 - Current best internal CV score: 8137.9527071


Optimization Progress:  78%|███████▊  | 94/120 [1:20:15<58:37, 135.30s/pipeline]ptimization Progress:  69%|██████▉   | 83/120 [1:05:11<27:11, 44.09s/pipeline]          Optimization Progress:  78%|███████▊  | 94/120 [1:20:15<58:37, 135.30s/pipeline]          

Timeout during evaluation of pipeline #94. Skipping to the next pipeline.
Generation 5 - Current best internal CV score: 8137.9527071


                                                                                Optimization Progress:  86%|████████▌ | 103/120 [1:22:18<21:54, 77.35s/pipeline]


Best pipeline: RandomForestRegressor(VarianceThreshold(input_matrix, 47.0))
618.937349758


In [13]:
dset

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour,weekday,h_wd
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,0,5,500
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,1,5,501
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,5,502
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,5,503
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,4,5,504
2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,5,5,505
2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,6,5,506
2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,7,5,507
2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,8,5,508
